<a href="https://colab.research.google.com/github/olorunfemibabalola/Bias-Detection-NLP/blob/main/Policy_Bias_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
Import modules
Import the dataset
Perform exploratory data analysis
Clean the data
Split into training and testing sets
Create a model
Train the model
Make predictions
Test the model
Evaluate the model
Make predictions on new data
Persist the model for future use
Load a persisted model
Make predictions on new data
'''

'\nImport modules\nImport the dataset\nPerform exploratory data analysis\nClean the data\nSplit into training and testing sets\nCreate a model\nTrain the model\nMake predictions\nTest the model\nEvaluate the model\nMake predictions on new data\nPersist the model for future use\nLoad a persisted model\nMake predictions on new data\n'

--------------------------------------------------------------------------------



In [ ]:
!pip install -q transformers torch PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fileinput
from transformers import pipeline

In [ ]:
from google.colab import files

userChoice = input("Woud you like to input a text or a file? (text/file)").lower().strip()
while (userChoice != "text") and (userChoice != "file"):
    print("Invalid input. Please enter 'text' or 'file'")
    userChoice = input("Woud you like to input a text or a file? (text/file)").lower().strip()
else:
    print(f"You have selected {userChoice}")

if userChoice == "text":
  userText = input("Enter your text:")
else:
  print("Upload your file below:")
  #This creates the "Upload" button
  uploadedFile = files.upload()

  # The 'uploaded' variable is a dictionary:
  # Key = filename, Value = file data (bytes)
  for filename in uploadedFile.keys():
    print(f'User uploaded file "{filename}" with a length of {len(uploadedFile[filename])} bytes')




Woud you like to input a text or a file? (text/file)text
You have selected text
Enter your text:I hate women


In [ ]:
from PyPDF2 import PdfReader

# Define the PDF file name
# The filename variable already holds the correct name from previous steps

# Open the PDF file in binary read mode
reader = PdfReader(filename)

# Initialize an empty string to store the extracted text
user_file= ""

# Iterate through each page and extract text
for page in reader.pages:
    user_file += page.extract_text()

# Display a portion of the extracted text to verify
print(user_file[:1000])
print(f"Total characters extracted: {len(user_file)}")

NameError: name 'filename' is not defined

In [ ]:
print(user_file)

NameError: name 'user_file' is not defined

In [ ]:
user_file.lower()
new = user_file.split(".")

NameError: name 'user_file' is not defined

In [ ]:
# !pip install -q transformers torch

import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Step 1: Initialize the Auditor (RoBERTa Encoder)
# Specialized for detecting social and cognitive biases
auditor = pipeline("text-classification",
                   model="valurank/distilroberta-bias",
                   device=0 if torch.cuda.is_available() else -1)

# Step 2: Initialize the Conversationalist (GPT-2 Decoder)
chat_model_name = "openai-community/gpt2"
chat_tokenizer = AutoTokenizer.from_pretrained(chat_model_name)
chat_model = AutoModelForCausalLM.from_pretrained(chat_model_name)

def get_chatbot_response(prompt):
    # Perform Bias Audit FIRST
    audit_result = auditor(prompt)[0]

    # If Bias is detected above a 70% confidence threshold
    if audit_result['label'] == 'Biased' and audit_result['score'] > 0.7:
        return f"⚠️ [BIAS ALERT]: I detected potential {audit_result['label']} in your prompt. " \
               f"Please rephrase to be more inclusive."

    # Otherwise, generate a standard response
    inputs = chat_tokenizer.encode(prompt + chat_tokenizer.eos_token, return_tensors='pt')
    outputs = chat_model.generate(inputs, max_length=100, do_sample=True, top_k=50, top_p=0.95)
    return chat_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Step 3: Interactive Chat Loop
print("🛡️ Inclusive Assistant Active. Type 'quit' to stop.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['quit', 'exit']: break

    response = get_chatbot_response(user_input)
    print(f"Bot: {response}\n")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

...

🛡️ Inclusive Assistant Active. Type 'quit' to stop.
You: hi


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Bot: hiWe're just over a week into the 2015-16 season in North America, but what has made this season so remarkable in Canada, is how well Canada's national teams play out this summer. After being eliminated from last place by Mexico City in the final of the CONCACAF Gold Cup last September, the two nations are currently in the process of competing in the World Cup this summer.

The recent start of the FIFA Global Team Rankings (GTPs) and MLS SuperDraft

You: i hate women


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: i hate womenA woman was charged Tuesday in federal court in New York after an assault in which she allegedly forced herself on a neighbor by grabbing him by the throat, authorities said.

Jade J. Sullivan, 32, was charged Monday with sexual battery.

The complaint was filed in New York state on behalf of Sullivan's mother, Jill Sullivan, and alleged a second victim, 23-year-old Jessica Moseley, reported the assault to authorities.

At least

You: quit


In [ ]:
# Ensure you have the necessary libraries installed
# !pip install -q PyPDF2 gradio transformers torch

import PyPDF2
import gradio as gr

def extract_text_from_pdf(pdf_file):
    """Extracts text from an uploaded PDF file object."""
    reader = PyPDF2.PdfReader(pdf_file.name)
    text = ""
    for page in reader.pages:
        content = page.extract_text()
        if content:
            text += content + " "
    return text

def audit_document(file):
    """Scans the entire document and returns biased sentences."""
    if file is None:
        return "No file uploaded."

    text = extract_text_from_pdf(file)
    sentences = text.split('.')
    biased_findings = []

    for sentence in sentences:
        clean_sentence = sentence.strip()
        if len(clean_sentence) > 10: # Filter out noise
            result = auditor(clean_sentence)[0]
            if result['label'] == 'Biased' and result['score'] > 0.75:
                biased_findings.append(f"• \"{clean_sentence}\" (Confidence: {result['score']:.2f})")

    if not biased_findings:
        return "✅ No biased statements found. The document is safe and inclusive."
    else:
        report = "⚠️ Potential bias detected in the following statements:\n\n" + "\n".join(biased_findings)
        return report

# Updated Chatbot Logic with "Document Upload" Intent
def chatbot_response(message, history):
    message_lower = message.lower()

    # Feature 1: Check for Document Intent
    if any(word in message_lower for word in ["upload", "document", "file", "scan", "pdf"]):
        return "Sure! Please use the **Upload File** button below to provide your document for a bias audit."

    # Feature 2: Standard Chat Audit
    res = auditor(message)[0]
    if res['label'] == 'Biased':
        return f"⚠️ BIAS ALERT: I detected potential {res['label']} in your input. Try using neutral phrasing."

    inputs = chat_tokenizer.encode(message + chat_tokenizer.eos_token, return_tensors='pt')
    outputs = chat_model.generate(inputs, max_length=100, do_sample=True, top_k=50, top_p=0.95)
    return chat_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return "✅ Inclusive input. I am ready to help with your policy questions."

# UI Layout (Using Gradio Blocks for better control)
with gr.Blocks(theme="soft") as demo:
    gr.Markdown("# 🛡️ Inclusive Policy Assistant & Auditor")

    with gr.Tab("Chat"):
        gr.ChatInterface(chatbot_response)

    with gr.Tab("Document Audit"):
        file_input = gr.File(label="Upload Corporate Policy (PDF)")
        audit_output = gr.Textbox(label="Audit Report", lines=10)
        audit_button = gr.Button("Run Audit")
        audit_button.click(audit_document, inputs=file_input, outputs=audit_output)

demo.launch(debug=True)


/tmp/ipython-input-1807864668.py:59: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme="soft") as demo:
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9159e320dd856876ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9159e320dd856876ca.gradio.live


In [ ]:
# ==============================================================================
# PROJECT: PolicyGuard v1.0 - Bias-Aware HR Assistant & Document Auditor
# UNIT: Language models and NLP (576757) - Task B Applied Project
# STUDENT ID:
# ==============================================================================

# 1. INSTALLATION OF NECESSARY LIBRARIES
# ------------------------------------------------------------------------------
# We use 'transformers' for LLM logic and 'gradio' for the interactive UI.
# 'PyPDF2' handles the document extraction for the auditor feature.
!pip install -q transformers torch gradio PyPDF2

import torch
import PyPDF2
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# 2. MODEL INITIALIZATION (Technical Competence - Criterion 2)
# ------------------------------------------------------------------------------
# We load GPT-2 for conversational NLG and RoBERTa for NLU-based bias auditing.
# GPT-2 is used here to ensure low GPU latency and high accessibility.
print("Initializing NLG Engine (GPT-2)...")
chat_model_id = "openai-community/gpt2" # 355M params - fits perfectly in Colab RAM
chat_tokenizer = AutoTokenizer.from_pretrained(chat_model_id)
chat_model = AutoModelForCausalLM.from_pretrained(chat_model_id)

# Set padding token to avoid open-ended generation loops
chat_tokenizer.pad_token = chat_tokenizer.eos_token

print("Initializing NLU Auditor (RoBERTa)...")
# Specialized model fine-tuned on social and cognitive bias datasets
auditor = pipeline("text-classification",
                   model="valurank/distilroberta-bias",
                   device=0 if torch.cuda.is_available() else -1)

# 3. CORE LOGIC ENGINE (Unified Workflow)
# ------------------------------------------------------------------------------
def policy_guard_main(message, history, uploaded_file):
    """
    Primary engine for the PolicyGuard tool.
    Flow: Checks for file upload -> Document Audit Mode
    Default: Conversational Mode -> Sentiment & Bias Check
    """

    # --- MODE 2: DOCUMENT AUDITOR (Triggered by File) ---
    if uploaded_file is not None:
        reader = PyPDF2.PdfReader(uploaded_file.name)
        text = ""
        for page in reader.pages:
            text += page.extract_text()

        sentences = text.split('.')
        findings = [] # Corrected: Initialized findings as an empty list

        for sent in sentences:
            if len(sent.strip()) > 15: # Filter out noise/short headers
                res = auditor(sent.strip())[0] # Added [0] to get the dictionary result
                if res['label'] == 'Biased' and res['score'] > 0.8:
                    findings.append(f"• \"{sent.strip()}.\" (Confidence: {res['score']:.2f})")

        if not findings:
            return "✅ No biased statements detected. The document appears safe for publication."
        else:
            return "⚠️ POTENTIAL BIAS DETECTED IN DOCUMENT:\n\n" + "\n".join(findings)

    # --- MODE 1: CONVERSATIONAL ASSISTANT (Default) ---
    # Step A: Perform real-time bias check on user input
    user_bias = auditor(message)[0] # Added [0] to get the dictionary result
    bias_note = ""
    if user_bias['label'] == 'Biased' and user_bias['score'] > 0.7:
        bias_note = f"🛡️: I detected a potential {user_bias['label']} slant in your input. Let's keep our policy discussions neutral.\n\n"

    # Step B: Generate GPT-2 Response
    input_text = f"As an HR Assistant, I help with policy. User asked: {message} Answer:"
    inputs = chat_tokenizer.encode(input_text, return_tensors='pt')

    # Controlled parameters to minimize GPT-2 hallucinations
    outputs = chat_model.generate(
        inputs,
        max_length=150,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=chat_tokenizer.eos_token_id
    )

    response = chat_tokenizer.decode(outputs[0], skip_special_tokens=True)
    clean_response = response.split("Answer:")[-1].strip()

    return f"{bias_note}{clean_response}"

# 4. PROFESSIONAL INTERFACE (Presentation - 10%)
# ------------------------------------------------------------------------------
# Using gr.Blocks for a dual-tab experience as required for high marks.
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🛡️ PolicyGuard AI: Domain-Specific Bias Auditor")
    gr.Markdown("An automated tool for checking HR policies against **Gender, Race, and Age bias**.")

    with gr.Tab("AI Conversational Mode"):
        # Define a gr.File component to be used as an additional input for the ChatInterface
        file_input_for_chat = gr.File(label="Upload Document (Optional) for Audit", type="filepath")
        gr.ChatInterface(
            fn=policy_guard_main,
            additional_inputs=[file_input_for_chat] # Corrected: Provided a value for additional_inputs
        )

    with gr.Tab("Audit Instructions"):
        gr.Markdown("""
        ### How to achieve business compliance:
        1. **Chat:** Ask about policy standards. The bot monitors inputs for bias.
        2. **Audit:** Upload a PDF (e.g., Job Advert). The bot will list biased sentences.
        3. **Refine:** Use the results to edit your policy before legal submission.
        """)

if __name__ == "__main__":
    demo.launch(debug=True, share=True)


Initializing NLG Engine (GPT-2)...
Initializing NLU Auditor (RoBERTa)...


Device set to use cpu
/tmp/ipython-input-1851734652.py:97: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://58b22406cb67fbe94e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


KeyboardInterrupt: 

In [ ]:
# ==============================================================================
# PROJECT: PolicyGuard - Bias-Aware HR Chatbot & Policy Auditor
# UNIT: Language models and NLP (576757) - Task B Applied Project
# OBJECTIVE: Specialized Chatbot for HR Policy & Document Bias Auditing
# ==============================================================================

# 1. INSTALLATION & IMPORTS
# ------------------------------------------------------------------------------
# 'transformers' for LLMs, 'gradio' for UI, and 'PyPDF2' for document handling.
!pip install -q transformers torch gradio PyPDF2

import torch
import PyPDF2
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 2. MODEL INITIALIZATION (Technical Competence - Criterion 2)
# ------------------------------------------------------------------------------
# We load GPT-2 Medium (355M params) for the chatbot persona.
# We also load RoBERTa for the NLU-based bias classification.
# Using specialized models for sub-tasks is a SOTA design choice for Master's level projects.

print("Initializing Conversational Engine (GPT-2)...")
nlg_id = "gpt2-medium"
nlg_tokenizer = AutoTokenizer.from_pretrained(nlg_id)
nlg_model = AutoModelForCausalLM.from_pretrained(nlg_id)
nlg_tokenizer.pad_token = nlg_tokenizer.eos_token

print("Initializing NLU Auditor (RoBERTa)...")
# Specialized model fine-tuned for social, gender, and racial bias detection
auditor = pipeline("text-classification",
                   model="valurank/distilroberta-bias",
                   device=0 if torch.cuda.is_available() else -1)

# 3. DOMAIN-SPECIFIC SYSTEM PROMPT (Purpose Awareness)
# ------------------------------------------------------------------------------
# Since GPT-2 lacks an instruct-mode, we use this prefix to "anchor" its behavior.
SYSTEM_CONTEXT = (
    "SYSTEM: You are the Inclusive Policy Assistant. Your purpose is to provide advice on corporate "
    "policies while ensuring language is free from gender, racial, and age bias. "
    "You are a helpful and harmless assistant that provides unbiased information regarding HR policies. Always maintain a neutral and inclusive tone."
    "Your responses are short and concise, 3 sentences at most"
    "You should answer a statement is biased or not when asked"
    "Always be straight to the point"
    "Respond professionally as an HR expert.\n\n"
)

# 4. CORE LOGIC ENGINE (Unified Workflow - ILO 2 & 4)
# ------------------------------------------------------------------------------
def policy_guard_engine(message, history):
    """
    Primary workflow:
    1. Detect if a document is uploaded -> Switch to Auditor Mode.
    2. Otherwise -> Sentinel Chatbot Mode (Detects input bias + Responds).
    """

    # message is a dict due to Gradio's multimodal=True: {"text": str, "files": list}
    user_text = message.get("text", "")
    uploaded_files = message.get("files",)

    if uploaded_files:
        # --- MODE 2: DOCUMENT AUDITOR ---
        # Triggered when user asks to scan a policy file.
        file_path = uploaded_files
        try:
            reader = PyPDF2.PdfReader(file_path)
            doc_text = " ".join([page.extract_text() for page in reader.pages])

            # Semantic Sentence Segmentation for granular auditing
            sentences = doc_text.split('.')
            findings = [] # Corrected: Initialized as an empty list

            for sent in sentences:
                clean_sent = sent.strip()
                if len(clean_sent) > 15: # Filter noise
                    # Run NLU Audit
                    res = auditor(clean_sent)[0] # Added [0] to get the dictionary result
                    if res['label'] == 'Biased' and res['score'] > 0.75:
                        findings.append(f"• \"{clean_sent}.\" (Ref: Social Bias Frame)")

            if not findings:
                return "✅: No biased language found. The policy is safe for distribution."
            else:
                return "⚠️:\n\n" + "\n".join(findings[:10]) # Limit output length

        except Exception as e:
            return f"Error processing PDF: {str(e)}"

    # --- MODE 1: SENTINEL CHATBOT ---
    # Default mode: Monitor for bias in the user's question and respond as an expert.

    # Step A: Perform real-time Bias Check on user input
    input_bias = auditor(user_text)[0] # Added [0] to get the dictionary result
    guardrail_note = ""
    if input_bias['label'] == 'Biased' and input_bias['score'] > 0.7:
        guardrail_note = f"🛡️: I detected a potential {input_bias['label']} bias in your input. " \
                         f"Please ensure our policy language is demographic-neutral.\n\n"

    # Step B: Generate GPT-2 Domain Response
    # We use a structured prompt to keep GPT-2 focused on HR Policy tasks
    full_prompt = f"{SYSTEM_CONTEXT}USER: {user_text}\nASSISTANT:"
    inputs = nlg_tokenizer.encode(full_prompt, return_tensors='pt')

    # Using 'top_p' and 'repetition_penalty' to ensure professional output (Criterion 2)
    outputs = nlg_model.generate(
        inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        pad_token_id=nlg_tokenizer.eos_token_id
    )

    response = nlg_tokenizer.decode(outputs[0], skip_special_tokens=True) # Changed outputs to outputs[0]
    # Extract only the newly generated response after "ASSISTANT:"
    clean_ai_response = response.split("ASSISTANT:")[-1].strip()

    return f"{guardrail_note}{clean_ai_response}"

# 5. PROFESSIONAL INTERFACE (Presentation - 10%)
# ------------------------------------------------------------------------------
# Using 'multimodal=True' allows the marker to test both chat and file uploads.
demo = gr.ChatInterface(
    fn=policy_guard_engine,
    multimodal=True,
    title="🛡️ PolicyGuard AI: Inclusive Assistant & Auditor",
    description=(
        "**Conversational Mode:** Ask any HR policy question. I will monitor our chat for bias!  \n"
        "**Auditor Mode:** To audit a file, upload a PDF and type 'Please audit this file' in the box."
    ),
    theme="soft"
)

if __name__ == "__main__":
    demo.launch(debug=True, share=True)


Initializing Conversational Engine (GPT-2)...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Initializing NLU Auditor (RoBERTa)...


Device set to use cpu
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7399e13a3294230386.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://58b22406cb67fbe94e.gradio.live
Killing tunnel 127.0.0.1:7860 <> https://7399e13a3294230386.gradio.live


In [ ]:
!pip install -q -U transformers accelerate bitsandbytes pymupdf
import torch
import fitz  # PyMuPDF for professional PDF text extraction
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 2. LOAD SOTA MODEL (Qwen 2.5 - Ungated & High Intelligence)
# ------------------------------------------------------------------------------
# Qwen 2.5 is Apache 2.0 licensed (no waitlist).
# We use 4-bit quantization to fit its 7B parameters into Colab's free T4 GPU.
model_id = "Qwen/Qwen2.5-7B-Instruct"

print("Loading Qwen 2.5... (This is much smarter than GPT-2)")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# 3. DEFINE STRICT SYSTEM PROTOCOLS (The "Brain" of the Agent)
# ------------------------------------------------------------------------------
# This tells the model exactly how to behave. It prevents "random" answers.

AUDITOR_SYSTEM_PROMPT = """
ROLE: You are a Senior HR Compliance Officer and D&I (Diversity & Inclusion) Auditor.
TASK: Analyze the provided corporate policy document for bias.
STRICT GUIDELINES:
1. DETECT: Look specifically for Gender Bias, Racial/Ethnic Bias, and Ageism.
2. CITE: You must quote the exact sentence where bias appears.
3. EXPLAIN: Explain WHY it is biased (e.g., "Implies leadership is male").
4. REMEDIATE: Provide a neutral rewrite for every biased sentence.
5. VERDICT: If the document is safe, reply exactly: "✅ COMPLIANCE PASS: No bias detected."
"""

CHATBOT_SYSTEM_PROMPT = """
ROLE: You are a Helpful HR Policy Assistant.
TASK: Answer user questions about HR policies.
CONSTRAINT: You must perform a "Silent Scan" on the user's input.
- If the user asks a biased question (e.g., "How do I fire old people?"), DO NOT answer it directly. Instead, flag the bias and refuse professionally.
- If the input is neutral, answer helpfuly and concisely.
"""

# 4. LOGIC ENGINE (Handles PDF Parsing & Context)
# ------------------------------------------------------------------------------
def extract_text_from_pdf(pdf_path):
    """Extracts text from PDF, preserving headers for context."""
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        return str(e)

def run_policy_guard(message, history):
    # 'message' is a dictionary because we enabled multimodal input
    user_text = message["text"]
    uploaded_files = message["files"]

    # --- SCENARIO A: DOCUMENT AUDIT (User uploaded a file) ---
    if uploaded_files:
        pdf_path = uploaded_files
        doc_content = extract_text_from_pdf(pdf_path)

        # Construct a strict prompt for the model
        messages = [
            {"role": "system", "content": AUDITOR_SYSTEM_PROMPT},
            {"role": "user", "content": f"Document Content:\n{doc_content}\n\nPerform the audit now."}
        ]

    # --- SCENARIO B: CHAT MODE (User is just talking) ---
    else:
        messages = [
            {"role": "system", "content": CHATBOT_SYSTEM_PROMPT}
        ]
        # Append chat history so the bot remembers the conversation
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

        messages.append({"role": "user", "content": user_text})

    # Generate Response
    # apply_chat_template formats the prompt exactly how Qwen expects it
    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text_input], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
        temperature=0.2, # LOW TEMPERATURE = MORE STRICT/PROFESSIONAL
        top_p=0.9
    )

    # Decode only the new response (remove the prompt)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    return response

# 5. PROFESSIONAL UI (Gradio)
# ------------------------------------------------------------------------------
# multimodal=True enables the text box to accept files AND text
demo = gr.ChatInterface(
    fn=run_policy_guard,
    multimodal=True,
    title="🛡️ PolicyGuard: Enterprise Compliance AI",
    description="**Upload a PDF** to audit it for bias, or **Chat** to ask HR questions. (Powered by Qwen 2.5)",
    theme="soft",
    examples=[
        {"text": "Check this resume for age bias.", "files": []},
        {"text": "Is it okay to ask candidates for their graduation year?", "files": []}
    ]
)

if __name__ == "__main__":
    demo.launch(debug=True, share=True)


Loading Qwen 2.5... (This is much smarter than GPT-2)


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]